# Dependance of O2 wrt airmass

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2024/11/24
- last update : 2024/12/03


This emulator is based from datagrid of atmospheric transparencies extracted from libradtran

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# to enlarge the sizes
params = {'legend.fontsize': 'large',
          'figure.figsize': (10, 6),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
plt.rcParams.update(params)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Imports

In [ ]:
from getObsAtmo.getObsAtmo import ObsAtmo,validateObsName,Dict_Of_sitesPressures,getObsSiteDataFrame

# Check information

In [ ]:
Dict_Of_sitesPressures

### Choose Rubin-LSST observation sites but other sites may be chosen

In [ ]:
obssitename = 'LSST'
obs_str = validateObsName(obssitename) 
std_pressure = Dict_Of_sitesPressures[obs_str]

In [ ]:
emul =  ObsAtmo(obs_str= obssitename)

# Typical transmission plot

- show a typical transmission plot

In [ ]:
emul.plot_transmission()

# Setting atmospheric parameters

In [ ]:
WL = emul.GetWL()
airmasses = emul.AIRMASS
NAM = len(airmasses)
pwvs = emul.PWV
NPWV = len(pwvs)
ozs = emul.OZ
NOZ = len(ozs)

In [ ]:
airmasses

In [ ]:
hls_colors = sns.color_palette("hls", NAM)
sns.palplot(hls_colors)

## Check emulator with O2 absorption

Expect no difference

In [ ]:
hls_colors = sns.color_palette("hls", NPWV)
sns.palplot(hls_colors)

In [ ]:
am =1

fig, ax = plt.subplots()
for idx,pwv in enumerate(pwvs):

    transm = emul.GetO2absTransparencyArray(WL,am,pwv)
  
    if idx == 0: 
        ax.plot(WL,transm,"-",color=hls_colors[idx])
    else: 
        ax.plot(WL,transm,"-",color=hls_colors[idx])
        
title = f"ObsAtmo : O2 absorption for different pressures, am = {am:.1f}"
ax.set_title(title)
ax.set_xlabel("$\lambda$ (nm)")
ax.set_ylabel("transmission")
ax.set_xlim(680.,780.)
# place a text box in upper left in axes coords
#ax.text(0.5, 0.05, textboxstr, transform=ax.transAxes, fontsize=14,verticalalignment='bottom', bbox=textboxprops)
ax.grid()
ax.legend()

ax.axvline(687.0,color="k")
ax.axvline(689.0,color="k")
ax.axvline(759.0,color="k")
ax.axvline(760.0,color="k")
ax.axvline(763.0,color="k")
ax.axvline(765.0,color="k")
ax.axvline(767.0,color="k")

plt.show()

## Dependence with airmass

In [ ]:
WLtest = np.array([687,689,759.,760,763.,765,767])
NWLtest = len(WLtest)
#hls_colors = sns.color_palette("Spectral_r", NWLtest)
hls_colors = sns.color_palette("coolwarm",NWLtest)
#sns.palplot(hls_colors)
sns.palplot(hls_colors)

In [ ]:
binnumber = np.digitize(WLtest,WLtest)
binnumber

In [ ]:
linestyles = ["-","--","-.",":",  "-","--","-.",":"]
linewidths = [4,4,4,4,2,2,2,2]

## Plot

In [ ]:
fig,ax = plt.subplots(1,1)

# compute a new transmission array in airmass x wl
transmarray = np.zeros((NAM,NWLtest))
    
for idx_am,airmass in enumerate(airmasses):
    transmarray[idx_am,:]  = emul.GetO2absTransparencyArray(WLtest,airmass)
    # normalize wrt airmass = 1 for each wavelength
    #norm_am0 = np.copy(transmarray[0,:])
    
    # renormalize to first airmass
    #for idx_am,airmass in enumerate(airmasses):
    #    transmarray[idx_am,:] =  transmarray[idx_am,:]/norm_am0

    # plot attenuation 
for idx_wl,wl in enumerate(WLtest):
    x = airmasses
    y = transmarray[:,idx_wl]
        
    bin_number_for_wl = np.digitize(wl,WLtest)-1
    the_wl_color = hls_colors[bin_number_for_wl]
    ls = linestyles[0]
    lw = linewidths[0]

    
    label = "$\\lambda = $" + f"{wl:.0f} nm"
    ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw,label=label)
        
#ax.legend(loc='upper right')
ax.legend(bbox_to_anchor=(1.05, 1.01))
ax.set_xlabel("$airmass$")  
ax.set_ylabel("$T_{O2}$")  
ax.set_title("Di-Oxygen Air transmission vs $airmass$")

In [ ]:
fig,ax = plt.subplots(1,1)

# compute a new transmission array in airmass x wl
transmarray = np.zeros((NAM,NWLtest))
    
for idx_am,airmass in enumerate(airmasses):
    transmarray[idx_am,:]  = emul.GetO2absTransparencyArray(WLtest,airmass)
    # normalize wrt airmass = 1 for each wavelength
    #norm_am0 = np.copy(transmarray[0,:])
    
    # renormalize to first airmass
    #for idx_am,airmass in enumerate(airmasses):
    #    transmarray[idx_am,:] =  transmarray[idx_am,:]/norm_am0

    # plot attenuation 
for idx_wl,wl in enumerate(WLtest):
    x = airmasses
    y = -2.5*np.log10(transmarray[:,idx_wl])
        
    bin_number_for_wl = np.digitize(wl,WLtest)-1
    the_wl_color = hls_colors[bin_number_for_wl]
    ls = linestyles[0]
    lw = linewidths[0]

    
    label = "$\\lambda = $" + f"{wl:.0f} nm"
    ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw,label=label)
        
#ax.legend(loc='upper right')
ax.legend(bbox_to_anchor=(1.05, 1.01))
ax.set_xlabel("$airmass$")  
ax.set_ylabel("$-2.5\\log_{10}(1-T_{O2})$ (mag)")  
ax.set_title("Di-Oxygen extinction vs $airmass$")

In [ ]:
fig,ax = plt.subplots(1,1)

# compute a new transmission array in airmass x wl
transmarray = np.zeros((NAM,NWLtest))
    
for idx_am,airmass in enumerate(airmasses):
    transmarray[idx_am,:]  = emul.GetO2absTransparencyArray(WLtest,airmass)
    # normalize wrt airmass = 1 for each wavelength
    #norm_am0 = np.copy(transmarray[0,:])
    
    # renormalize to first airmass
    #for idx_am,airmass in enumerate(airmasses):
    #    transmarray[idx_am,:] =  transmarray[idx_am,:]/norm_am0

    # plot attenuation 
for idx_wl,wl in enumerate(WLtest):
    x = np.log10(airmasses)
    y = np.log10(1-transmarray[:,idx_wl])
        
    bin_number_for_wl = np.digitize(wl,WLtest)-1
    the_wl_color = hls_colors[bin_number_for_wl]
    ls = linestyles[0]
    lw = linewidths[0]

    
    label = "$\\lambda = $" + f"{wl:.0f} nm"
    ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw,label=label)
        
#ax.legend(loc='upper right')
ax.legend(bbox_to_anchor=(1.05, 1.01))
ax.set_xlabel("$\\log_{10}(airmass)$")  
ax.set_ylabel("$-2.5\\log_{10}(T_{O2})$ (mag)")  
ax.set_title("Di-Oxygen extinction vs $airmass$")

## Fit

In [ ]:
xfit = np.linspace(0,3.,100)


fig,ax = plt.subplots(1,1)

# compute a new transmission array in airmass x wl
transmarray = np.zeros((NAM,NWLtest))
    
for idx_am,airmass in enumerate(airmasses):
    transmarray[idx_am,:]  = emul.GetO2absTransparencyArray(WLtest,airmass)
    # normalize wrt airmass = 1 for each wavelength
    #norm_am0 = np.copy(transmarray[0,:])
    
    # renormalize to first airmass
    #for idx_am,airmass in enumerate(airmasses):
    #    transmarray[idx_am,:] =  transmarray[idx_am,:]/norm_am0

    # plot attenuation 
for idx_wl,wl in enumerate(WLtest):
    x = airmasses
    y = -2.5*np.log10(transmarray[:,idx_wl])
        
    bin_number_for_wl = np.digitize(wl,WLtest)-1
    the_wl_color = hls_colors[bin_number_for_wl]
    ls = linestyles[0]
    lw = linewidths[0]

    z = np.polyfit(x, y, deg=1)
    p = np.poly1d(z)

    # slope and constant in mmag
    gamma = z[0]*1000.  #mmag/DU
    const = z[1]*1000.  #mmag

    label = "$\\lambda$ = "+f"{wl:.0f} nm "+ f"fit : slope = {gamma:.3f} mmag/airmass const = {const:.3f} mmag"

    
    ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw,label=label)
    ax.plot(xfit,p(xfit),':',color="grey")
        
#ax.legend(loc='upper right')
ax.legend(bbox_to_anchor=(1.05, 1.01))
ax.set_xlabel("$airmass$")  
ax.set_ylabel("$-2.5\\log_{10}(T_{O2})$ (mag)")  
ax.set_title("Di-Oxygen extinction vs $airmass$")

In [ ]:
xfit = np.linspace(-0.5,0.5,100)

fig,ax = plt.subplots(1,1)

# compute a new transmission array in airmass x wl
transmarray = np.zeros((NAM,NWLtest))
    
for idx_am,airmass in enumerate(airmasses):
    transmarray[idx_am,:]  = emul.GetO2absTransparencyArray(WLtest,airmass)
    # normalize wrt airmass = 1 for each wavelength
    #norm_am0 = np.copy(transmarray[0,:])
    
    # renormalize to first airmass
    #for idx_am,airmass in enumerate(airmasses):
    #    transmarray[idx_am,:] =  transmarray[idx_am,:]/norm_am0

    # plot attenuation 
for idx_wl,wl in enumerate(WLtest):
    x = np.log10(airmasses)
    y = np.log10(1-transmarray[:,idx_wl])
        
    bin_number_for_wl = np.digitize(wl,WLtest)-1
    the_wl_color = hls_colors[bin_number_for_wl]
    ls = linestyles[0]
    lw = linewidths[0]

    z = np.polyfit(x, y, deg=1)
    p = np.poly1d(z)

    # slope and constant in mmag
    gamma = z[0]*1000.  #mmag/DU
    const = z[1]*1000.  #mmag

    label = "$\\lambda$ = "+f"{wl:.0f} nm "+ f"fit : slope = {gamma:.3f} mmag/log10(airmass) const = {const:.3f} mmag"


    ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw,label=label)
    ax.plot(xfit,p(xfit),':',color="grey")
        
#ax.legend(loc='upper right')
ax.legend(bbox_to_anchor=(1.05, 1.01))
ax.set_xlabel("$\\log_{10}(airmass)$")  
ax.set_ylabel("$-2.5\\log_{10}(T_{O2})$ (mag)")  
ax.set_title("Di-Oxygen extinction vs $airmass$")

## Fit Lambert function

In [ ]:
from scipy.special import wrightomega, lambertw
from scipy.optimize import curve_fit

def absorption_lambertw(x, a, b ):
    return np.real((a * lambertw(x*b ,0)))

In [ ]:
fig,ax = plt.subplots(1,1)

xfit = np.linspace(0.,3.,100)

# compute a new transmission array in airmass x wl

# loop on wavelength
for idx_wl,wl in enumerate(WLtest):

    transmarray = np.zeros(NAM)

    # loop on airmass
    for idx_am,airmass in enumerate(airmasses):
        transmarray[idx_am]  = emul.GetO2absTransparencyArray([wl],airmass)
       
    x = airmasses
    y = - 2.5*np.log10(transmarray)
        

    # Do the fit
    try:
        popt, pcov = curve_fit(absorption_lambertw, x, y,  bounds=(0, [3., 2.]), method = 'trf')
        yfit = absorption_lambertw(xfit,popt[0], popt[1] )
        ax.plot(xfit,yfit,':',color="grey")
    except:
        pass

    bin_number_for_wl = np.digitize(wl,WLtest)-1
    the_wl_color = hls_colors[bin_number_for_wl]
    ls = linestyles[0]
    lw = linewidths[0]

    label = "$\\lambda = $" + f"{wl:.0f} nm"

    index_sorted = np.argsort(x)
    
    ax.plot(x[index_sorted],y[index_sorted],color=the_wl_color,ls = ls,lw=lw,label=label)


#ax.legend(loc='lower right')
ax.legend(bbox_to_anchor=(1.05, 1.01))
ax.set_xlabel("$airmass$")  
ax.set_ylabel("$-2.5 \\log_{10}(T_{O2})$ (mag)")  
ax.set_title("O2Air extinction vs $airmass$ : Fit of Lambert W function")

plt.show()


In [ ]:
fig,ax = plt.subplots(1,1)

xfit = np.linspace(0.1,10.,100)

# compute a new transmission array in airmass x wl

# loop on wavelength
for idx_wl,wl in enumerate(WLtest):

    transmarray = np.zeros(NAM)

    # loop on airmass
    for idx_am,airmass in enumerate(airmasses):
        transmarray[idx_am]  = emul.GetO2absTransparencyArray([wl],airmass)
       
    x = airmasses
    y = - 2.5*np.log10(transmarray)
        

    # Do the fit
    try:
        popt, pcov = curve_fit(absorption_lambertw, x, y,  bounds=(0, [3., 2.]), method = 'trf')
        yfit = absorption_lambertw(xfit,popt[0], popt[1] )
        ax.plot(xfit,yfit,':',color="grey")
    except:
        pass

    bin_number_for_wl = np.digitize(wl,WLtest)-1
    the_wl_color = hls_colors[bin_number_for_wl]
    ls = linestyles[0]
    lw = linewidths[0]

    label = "$\\lambda = $" + f"{wl:.0f} nm"

    index_sorted = np.argsort(x)
    
    ax.plot(x[index_sorted],y[index_sorted],color=the_wl_color,ls = ls,lw=lw,label=label)


#ax.legend(loc='lower right')
ax.legend(bbox_to_anchor=(1.05, 1.01))
ax.set_xlabel("$airmass$")  
ax.set_ylabel("$-2.5 \\log_{10}(T_{O2})$ (mag)")  
ax.set_title("O2Air extinction vs $airmass$ : Fit of Lambert W function")

ax.set_xscale("log")
ax.set_yscale("log")

plt.show()
